In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import evaluate
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from dataclasses import dataclass


@dataclass
class Config:
    MODEL_PATH       = "./whisper-ja-asmr-small-1-earlyst/final"   # your fine-tuned model dir
    MANIFEST_CSV     = "./manifest/preprocessed-segments-index.csv"
    TEST_RATIO       = 0.1
    RANDOM_SEED      = 42
    DEVICE           = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Load & split out the test set
df = pd.read_csv(Config.MANIFEST_CSV)
df = df[df.lang == "ja"].reset_index(drop=True)
_, test_df = np.split(
    df.sample(frac=1, random_state=Config.RANDOM_SEED),
    [ int(len(df)*(1 - Config.TEST_RATIO)) ]
)
print(f"▶ Test examples: {len(test_df)}")

# 2) Prepare metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

# 3) Load model + processor, clear any forced prompts
processor = WhisperProcessor.from_pretrained(Config.MODEL_PATH)
model     = WhisperForConditionalGeneration.from_pretrained(Config.MODEL_PATH).to(Config.DEVICE)

▶ Test examples: 1698


/home/rehab/research/whisperlaz/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [62]:
#––– Before the loop, we’ve already set:
# model.generation_config.num_beams            = 3
# model.generation_config.no_repeat_ngram_size = 2
# model.generation_config.repetition_penalty    = 1.5
# model.generation_config.length_penalty        = 1.0
# model.generation_config.early_stopping        = True

# (You can also set:)
model.generation_config.max_new_tokens = 50  # generate at most 50 new tokens

#––– 4) Inference loop with max_new_tokens
records = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Eval with anti-repeat"):
    data      = np.load(row["npz_path"], allow_pickle=True)
    audio_arr = data["audio"].astype(np.float32)
    reference = str(data["text"])

    # tokenize + mask
    inputs = processor(
        audio_arr,
        sampling_rate=16000,
        return_tensors="pt",
        return_attention_mask=True
    )
    input_feats    = inputs.input_features.to(Config.DEVICE)
    attention_mask = inputs.attention_mask.to(Config.DEVICE)

    with torch.no_grad():
        gen_ids = model.generate(
            input_feats,
            attention_mask=attention_mask,
            # GENERATION-ONLY cap:
            max_new_tokens=50,

            # anti-repetition (redundant if you set on generation_config, but explicit is fine)
            num_beams=model.generation_config.num_beams,
            no_repeat_ngram_size=model.generation_config.no_repeat_ngram_size,
            repetition_penalty=model.generation_config.repetition_penalty,
            length_penalty=model.generation_config.length_penalty,
            early_stopping=model.generation_config.early_stopping,

            # ensure no old prompts
            forced_decoder_ids=None,
        )

    hypothesis = processor.batch_decode(gen_ids, skip_special_tokens=True)[0]

    records.append({
        "npz_path":   row["npz_path"],
        "reference":  reference,
        "prediction": hypothesis
    })

Eval with anti-repeat:   0%|          | 0/1698 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [37]:
# 5) Compute overall WER/CER
df_results = pd.DataFrame(records)
wer = wer_metric.compute(predictions=df_results.prediction, references=df_results.reference)
cer = cer_metric.compute(predictions=df_results.prediction, references=df_results.reference)
print(f"\nOverall  → WER: {wer:.3f}, CER: {cer:.3f}\n")

# 6) Show per-example DataFrame
df_results.head(10)


Overall  → WER: 1.070, CER: 0.560



,npz_path,reference,prediction
0,./preprocessed_segments/channel_1_K-wGFJzWXEM_...,なかなか大変だなって思いました,なかなか大変だなって思いました。
1,./preprocessed_segments/channel_1__nt_OsOMIXA_...,キーピングスタディーレコード はここに当てはまりそうですね,スタディーレコード ここに当てはあまりそうですね。
2,./preprocessed_segments/channel_1__nt_OsOMIXA_...,40番バクツ 2年生が書かれていてその上に当てはまる,40番パクツ 記念数が書かれていてその上に当てはまるものですけども、
3,./preprocessed_segments/channel_1__V7K7h1BPmk_...,毎日のように,毎日のように、
4,./preprocessed_segments/channel_1_K-wGFJzWXEM_...,文書1と文章には関連付けて考えてみ ようって話で,文章1と文書2を関連つけて考えてみようって話で 文座におかんれんづけて考え てみよってはなし...
5,./preprocessed_segments/channel_1_YExNnL0_WOQ_...,はい ではお部屋にご案内させていただきます,ではお部屋にご案内させていただきます。はい。
6,./preprocessed_segments/channel_1_aWCumaPnmIw_...,利益が全て自分の手元に入るという制度になります。,その利益が全て自分の手元に入るという制度になります。
7,./preprocessed_segments/channel_1__nt_OsOMIXA_...,これは,これは、はい。
8,./preprocessed_segments/channel_1_K-wGFJzWXEM_...,ある意味 復興を,ある意味 復興をになってもらいたいですけどね あるいみにふっこうを拭くことができるのかもしれ...
9,./preprocessed_segments/channel_1_oWq4pn4Lnxk_...,このように足・消化器官・脳が繋がっているように 体全体が繋がっているんですね,このように足証拠期間脳が繋がっているように体全体がつながってるんですね。


In [38]:
#%% 6) Listen to a few examples with their predictions
from IPython.display import Audio, display

# Pick the first 5 (or sample randomly: df_results.sample(5, random_state=42))
for idx, row in df_results.head(5).iterrows():
    # Load raw waveform
    audio_arr = np.load(row["npz_path"], allow_pickle=True)["audio"].astype(np.float32)
    
    # Print out the texts
    print(f"▶ Example {idx}")
    print(f"   Reference : {row['reference']}")
    print(f"   Prediction: {row['prediction']}")
    
    # Play the audio
    display(Audio(audio_arr, rate=16000))
    print("-" * 60)


▶ Example 0
   Reference : なかなか大変だなって思いました
   Prediction: なかなか大変だなって思いました。


------------------------------------------------------------
▶ Example 1
   Reference : キーピングスタディーレコード はここに当てはまりそうですね
   Prediction: スタディーレコード ここに当てはあまりそうですね。


------------------------------------------------------------
▶ Example 2
   Reference : 40番バクツ 2年生が書かれていてその上に当てはまる
   Prediction: 40番パクツ 記念数が書かれていてその上に当てはまるものですけども、


------------------------------------------------------------
▶ Example 3
   Reference : 毎日のように
   Prediction: 毎日のように、


------------------------------------------------------------
▶ Example 4
   Reference : 文書1と文章には関連付けて考えてみ ようって話で
   Prediction: 文章1と文書2を関連つけて考えてみようって話で 文座におかんれんづけて考え てみよってはなして分からないんですけどね


------------------------------------------------------------
